In [54]:
import pickle
import yaml
import pandas as pd

In [59]:
def get_report(response):
    report = []
    for result in response.call_results:
        resp = result.parsed_response['results']
        for item in resp:
            item['file'] = result.files_evaluated[0] 
            report.append(item)
    return pd.DataFrame(report)

def extract_file_and_scores(resp_path):
    with open(resp_path, 'rb') as file:
        response = pickle.load(file)
    df = (
        get_report(response)
        .pivot(index='file', columns='ID', values='Score')
        .rename_axis(None, axis=1)
        .reset_index()
    )
    df['response_path'] = resp_path
    return df

In [58]:
with open('../data/processed/batch_run/record_combine.yml', 'r') as file:
    config = pd.DataFrame(yaml.safe_load(file))
config['response_path'].iloc[0]

'../data/processed/batch_run/lightfm_01.pickle'

In [64]:
res = [
    extract_file_and_scores(path) for path in config['response_path']
]
res = pd.concat(res, axis=0).reset_index(drop=True)

In [74]:
df_repo_run_file = config.merge(res, on='response_path', how='left')
df_repo_run_file

,repo,response_path,run,file,2.1,3.2,3.5,4.2,5.3,6.1,6.2
0,lightfm,../data/processed/batch_run/lightfm_01.pickle,1,../data/raw/openja/lightfm/tests/test_api.py,0.0,0.5,0.5,0.0,0.5,0.0,0.0
1,lightfm,../data/processed/batch_run/lightfm_01.pickle,1,../data/raw/openja/lightfm/tests/test_cross_va...,0.5,0.0,0.0,0.0,0.0,0.0,0.0
2,lightfm,../data/processed/batch_run/lightfm_01.pickle,1,../data/raw/openja/lightfm/tests/test_data.py,0.5,0.5,0.0,0.0,0.0,0.0,0.0
3,lightfm,../data/processed/batch_run/lightfm_01.pickle,1,../data/raw/openja/lightfm/tests/test_datasets.py,1.0,0.5,0.0,0.5,0.0,0.0,0.0
4,lightfm,../data/processed/batch_run/lightfm_01.pickle,1,../data/raw/openja/lightfm/tests/test_evaluati...,0.0,0.0,0.0,0.5,0.0,0.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1267,qlib,../data/processed/batch_run/qlib_06.pickle,6,../data/raw/openja/qlib/tests/test_get_data.py,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1268,qlib,../data/processed/batch_run/qlib_06.pickle,6,../data/raw/openja/qlib/tests/test_pit.py,1.0,0.0,0.0,0.0,0.0,0.5,0.0
1269,qlib,../data/processed/batch_run/qlib_06.pickle,6,../data/raw/openja/qlib/tests/test_register_op...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1270,qlib,../data/processed/batch_run/qlib_06.pickle,6,../data/raw/openja/qlib/tests/test_structured_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
checklist_ids = ['2.1', '3.2', '3.5', '4.2', '5.3', '6.1', '6.2']

In [102]:
df_repo_run = df_repo_run_file.groupby(['repo', 'run']).agg({
    id: ['max'] for id in checklist_ids
})
df_repo_run.columns = [col[0] for col in df_repo_run.columns]
df_repo_run = df_repo_run.reset_index()
df_repo_run

,repo,run,2.1,3.2,3.5,4.2,5.3,6.1,6.2
0,lightfm,1,1.0,0.5,0.5,0.5,0.5,1.0,1.0
1,lightfm,2,1.0,0.5,0.0,0.5,0.0,0.5,1.0
2,lightfm,3,1.0,0.5,1.0,0.5,0.5,0.5,0.5
3,lightfm,4,1.0,0.5,1.0,0.5,0.5,1.0,1.0
4,lightfm,5,1.0,0.5,0.0,0.5,0.0,0.5,1.0
5,lightfm,6,1.0,0.5,0.0,0.5,0.0,1.0,1.0
6,lightfm,7,1.0,0.5,0.0,0.5,0.0,0.5,1.0
7,lightfm,8,1.0,0.5,0.5,0.5,0.5,0.5,1.0
8,lightfm,9,1.0,0.5,0.0,0.5,0.0,1.0,1.0
9,lightfm,10,1.0,0.5,0.0,0.5,0.0,0.5,1.0


In [103]:
df_repo = df_repo_run.groupby(['repo']).agg({
    id: ['mean', 'std', 'count'] for id in checklist_ids
})
df_repo

2.1                       3.2                       3.5  \
             mean       std count      mean       std count      mean   
repo                                                                    
lightfm  0.983333  0.091287    30  0.483333  0.091287    30  0.266667   
qlib     1.000000  0.000000    10  0.850000  0.241523    10  0.000000   

                         4.2  ...         5.3                       6.1  \
             std count  mean  ... count  mean       std count      mean   
repo                          ...                                         
lightfm  0.38804    30  0.55  ...    30  0.20  0.249136    30  0.666667   
qlib     0.00000    10  0.50  ...    10  0.85  0.241523    10  0.900000   

                              6.2                  
              std count      mean       std count  
repo                                               
lightfm  0.239732    30  0.933333  0.172873    30  
qlib     0.210819    10  0.700000  0.258199    10  

[2 rows x 21 columns]

In [108]:
from collections import Counter
df_repo__count = df_repo_run.groupby(['repo'])['2.1'].apply(Counter).reset_index()
for id in checklist_ids[1:]:
    df_repo__count = df_repo__count.merge(
        df_repo_run.groupby(['repo'])[id].apply(Counter).reset_index(),
        on=['repo', 'level_1'],
        how='outer'
    )

df_repo__count = df_repo__count.fillna(0)
df_repo__count

,repo,level_1,2.1,3.2,3.5,4.2,5.3,6.1,6.2
0,lightfm,0.0,0.0,1.0,19.0,0.0,18.0,0.0,0.0
1,lightfm,0.5,1.0,29.0,6.0,27.0,12.0,20.0,4.0
2,lightfm,1.0,29.0,0.0,5.0,3.0,0.0,10.0,26.0
3,qlib,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
4,qlib,0.5,0.0,3.0,0.0,10.0,3.0,2.0,6.0
5,qlib,1.0,10.0,7.0,0.0,0.0,7.0,8.0,4.0


In [105]:
df_repo_run.groupby(['repo'])['2.1'].apply(Counter).reset_index()

,repo,level_1,2.1
0,lightfm,1.0,29.0
1,lightfm,0.5,1.0
2,qlib,1.0,10.0
3,qlib,0.5,NaN


In [110]:
import altair as alt
alt.__version__

'5.3.0'

In [ ]:
# the base chart
base = alt.Chart(source).transform_calculate(
    ymin="datum.y-datum.yerr",
    ymax="datum.y+datum.yerr"
)

# generate the points
points = base.mark_point(
    filled=True,
    size=50,
    color='black'
).encode(
    alt.X('x').scale(domain=(0, 6)),
    alt.Y('y').scale(zero=False)
)

# generate the error bars
errorbars = base.mark_errorbar().encode(
    x="x",
    y="ymin:Q",
    y2="ymax:Q"
)

points + errorbars